In [2]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize,LogNorm
import matplotlib.ticker as ticker
from matplotlib.cm import ScalarMappable
savefig = False
import importlib
import tango_model as tango_model
import sys
importlib.reload(sys.modules['tango_model'])
from tango_model import *
import scipy as sp
import torch
import networkx as nx

# Import the degree distribution CCDF from SEXUS
degree_dist_ccdf_sexus = np.genfromtxt('Data/Degree_dist_CCDF_SEXUS.csv', delimiter=',', skip_header=1)

In [3]:
N           = 85000
M           = 28
T           = 365*15
r_old_mean  = 0.2
pexp        = 1.55
lb          = 0.15
ub          = 70
window      = 365
old_dist    = "gamma"
alpha       = 2
T_track     = 365*10
n_couples   = N

simulation = EpidemicSimulation(N,M,r_old_mean,pexp,old_dist,alpha,ub,lb)

# Initialize peacetime parameters and run the simulation
simulation.initialize_peace_time(T=T, Twindow=window, progress=True, all_conn=False,T_track=T_track,n_couples=n_couples)
simulation.run_peace_time_simulation()

  3%|▎         | 170/5475 [00:09<04:55, 17.96it/s]


KeyboardInterrupt: 

In [ ]:
durations                   = np.array(simulation.get_partnership_durations(include_non_steady=True))
num_partnerships            = len(durations)
tot_encounters,tot_partners = simulation.analyze_encounters()
print(f"Number of partnerships: {num_partnerships}")

In [ ]:
active_mask         = tot_partners >= 1 
steadymask          = simulation.get_steady_partners_mask()

#print(f"Mean  and std number of partners:                                   {(tot_partners[active_mask]).float().mean():.2f} +- {(tot_partners[active_mask]).float().std():.2f} pr yr (SEXUS: 5.7 +- 9.2)")
print(f"Mean rate of encounters:                                            {(tot_encounters[active_mask]).float().mean()/52:.2f} pr wk (SEXUS: 0.74)")
print(f"Median rate of encounters:                                          {(tot_encounters[active_mask]).float().median()/52:.2f} pr wk (SEXUS: 0.5)")
#print(f"Fraction of people with exactly one partner the last year:          {(tot_partners == 1).sum()/(active_mask).sum():.2f} (SEXUS 0.41)")
print(f"Fraction of inactive:                                               {(tot_partners == 0).sum()/N:.2f} (SEXUS 0.12)")
print("")

print(f"Steady fraction:                                                    {sum(simulation.get_steady_partners_mask())/active_mask.sum():.2f}    (SEXUS 0.26)") #Don't divide by two, since we are counting members, not partnerships
print(f"Probability of relationship lasting a year:                         {(durations > 365).sum()/num_partnerships:.2f}    (SEXUS: 0.33)")

print("")

print(f"Mean number of yrly partners for members with partner:              {(tot_partners[active_mask &  steadymask]).float().mean():.2f} (SEXUS: 5.0 - (excluding partnerships newer than a year))")
print(f"Mean number of yrly partners for members without partner:           {(tot_partners[active_mask & ~steadymask]).float().mean():.2f} (SEXUS: 5.8)")
print("")
print(f"Mean  r_new among inactive:                                         {(simulation.r_new[~active_mask]).float().mean()*365:.2f} pr yr (total: {(simulation.r_new).float().mean()*365:.2f})")
print(f"Mean  r_old among inactive:                                         {(simulation.r_old[~active_mask]).float().mean()*7:.2f} pr wk (total: {(simulation.r_old).float().mean()*7:.2f})")

# Extract the active partners
active_partners = tot_partners[active_mask].numpy()

# Step 1: Define the specific x-axis values
x_values = np.array([1, 2, 3, 5, 10, 20, 50])

# Step 2: Calculate CCDF for only these x-axis values
ccdf = []
n = len(active_partners)
for value in x_values:
    # Probability that a value is greater than or equal to `value`
    prob = np.sum(active_partners >= value) / n
    ccdf.append(prob)

# Step 3: Plot CCDF for selected x-values
plt.figure(figsize=(8, 6))
plt.plot(x_values, ccdf, label='Simulation', marker='o')
plt.plot(x_values,degree_dist_ccdf_sexus, marker='o',label = "SEXUS")
plt.xticks(x_values)  # Show only the selected x-axis values
plt.xlabel('Active Partners')
plt.ylabel('P(X ≥ x)')
plt.xscale("log")
plt.yscale("log")
plt.title('Complementary Cumulative Distribution Function (CCDF)')
plt.grid(True,which = "both")
plt.legend()
plt.show()

In [ ]:
simulation.plot_sanity_check()

# Understanding the power law exponent

In [ ]:
# Does the KDist distribution itself give a different exponent, due to the low lower bound?
test_distribution = simulation.Kdist()
plt.hist(test_distribution[test_distribution > 1],bins = 100,label = "Simulation")
plt.yscale("log")
plt.xscale("log")
height = 5000
plt.plot([1,80],[height,height*80**-1.8],label = r"$\gamma = 1.8$")
plt.plot([1,80],[height,height*80**-1.55],label = r"$\gamma = 1.5$")
plt.legend()

In [ ]:
# Does the changed exponent continue farther up in the tail?

x_values2 = np.array([1, 2, 3, 5, 10, 20, 50,100,200])

# Step 2: Calculate CCDF for only these x-axis values
ccdf2 = []
n = len(active_partners)
for value in x_values2:
    # Probability that a value is greater than or equal to `value`
    prob = np.sum(active_partners >= value) / n
    ccdf2.append(prob)

# Step 3: Plot CCDF for selected x-values
plt.figure(figsize=(8, 6))
plt.plot(x_values2, ccdf2, label='Simulation', marker='o')
#plt.plot(x_values2,degree_dist_ccdf_sexus, marker='o',label = "SEXUS")
plt.plot([1,200],[1,200**(-0.8)])
plt.xticks(x_values2)  # Show only the selected x-axis values
plt.xlabel('Active Partners')
plt.ylabel('P(X ≥ x)')
plt.xscale("log")
plt.yscale("log")
plt.title('Complementary Cumulative Distribution Function (CCDF)')
plt.grid(True,which = "both")
plt.legend()
plt.show()